# AlvaroMLProject
## This is my final project for the Capstone Project for IBM Data Science



# Alvaro IBM Data Science Capstone 3 week task


### Hi! This is my final project for IBM Data Science Specialization. I will try to segment and cluster data from Belo Horizonte, a brazilian city. I will try to find the what is the better choice for a new food place

### First of all, we need to import the necessary librarys

In [1]:
#!conda install -c conda-forge geocoder --yes 
!pip install geocoder
!conda install -c conda-forge folium=0.5.0 --yes 
import requests # library to handle requests, necessary for retrieve information for the wikipedia page
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
from bs4 import BeautifulSoup # library to handle with the html and xml in Python

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import geocoder # Library to retrieve the latitute/longitude information from postal code # when you try to use geocoder

# Just some aditional imports to manipulate data and maps

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


import folium # map rendering library

    100% |████████████████████████████████| 102kB 14.9MB/s a 0:00:01
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    -

### Just some aditional imports to manipulate data and maps

In [2]:
# Second, we need to retrieve the data from GuiaMais page

In [3]:
#website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text # retrive the website
#soup = BeautifulSoup(website_url,'lxml') # process website using BeatifulSoup Library

In [4]:
website_url = requests.get('https://cep.guiamais.com.br/busca/savassi-belo+horizonte-mg').text # retrive the website
soup = BeautifulSoup(website_url,'lxml') # process website using BeatifulSoup Library
website_url = requests.get('https://cep.guiamais.com.br/busca/savassi-belo+horizonte-mg?page=2').text # retrive the website
soup2 = BeautifulSoup(website_url,'lxml') # process website using BeatifulSoup Library
website_url = requests.get('https://cep.guiamais.com.br/busca/savassi-belo+horizonte-mg?page=3').text # retrive the website
soup3 = BeautifulSoup(website_url,'lxml') # process website using BeatifulSoup Library

In [5]:
# Third, we need to find the table in data structure

In [6]:
My_table = soup.find('table',{'class':'table s_table_box table-striped table-responsive'})
My_table2 = soup2.find('table',{'class':'table s_table_box table-striped table-responsive'})
My_table3 = soup2.find('table',{'class':'table s_table_box table-striped table-responsive'})

In [7]:
# Next, we will iterate in the table, with HTML code, and process the data

In [8]:
trs = My_table.find_all('tr')[2:] # first we will find all the lines in the table
tds= [[0 for x in range(3)] for y in range(36)] # start one matrix. Is not a beatifull way to determine the lenght :/ but I am so tired 
i=0
for tr in trs: #next we need to iterate with all lines in the table , to process the data     
        line = tr.find_all('a')
        if tds[i-1][0] != line[0].text.rstrip():
            tds[i][0]= line[0].text.rstrip() #get the street name
            tds[i][1]= line[4].text.rstrip() #get the postal code 
            tds[i][2]= line[1].text.rstrip() #get the neighbourdhood
            i+=1 # then we will pass to the next line
trs = My_table2.find_all('tr')[2:] # first we will find all the lines in the table
for tr in trs: #next we need to iterate with all lines in the tacle , to process the data     
        line = tr.find_all('a')
        if tds[i-1][0] != line[0].text.rstrip():
            tds[i][0]= line[0].text.rstrip() #get the street name
            tds[i][1]= line[4].text.rstrip() #get the postal code 
            tds[i][2]= line[1].text.rstrip() #get the neighbourdhood
            i+=1 # then we will pass to the next line
trs = My_table3.find_all('tr')[2:] # first we will find all the lines in the table
for tr in trs: #next we need to iterate with all lines in the tacle , to process the data     
        line = tr.find_all('a')
        if tds[i-1][0] != line[0].text.rstrip():
            tds[i][0]= line[0].text.rstrip() #get the street name
            tds[i][1]= line[4].text.rstrip() #get the postal code 
            tds[i][2]= line[1].text.rstrip() #get the neighbourdhood
            i+=1 # then we will pass to the next line

In [9]:
df=pd.DataFrame(tds,columns=['Street', 'Postcode', 'Neighborhood']) # Then we need to transform the data into data frame
df

,Street,Postcode,Neighborhood
0,Rua Alagoas,30130-167,Savassi
1,Rua Antônio de Albuquerque,30112-010,Savassi
2,Rua Bernardo Guimarães,30140-081,Savassi
3,Avenida Brasil,30140-008,Savassi
4,Rua Ceará,30150-314,Savassi
5,Rua Cláudio Manoel,30140-105,Savassi
6,Avenida Cristóvão Colombo,30140-140,Savassi
7,Praça Diogo de Vasconcelos,30140-160,Savassi
8,Rua Fernandes Tourinho,30112-004,Savassi
9,Avenida Getúlio Vargas,30112-021,Savassi


In [10]:
# o que falta:
# criar um dataframe com os endereços dp jeito que eu printei
# chamara api do google pra trazer a latitude etc da savassi e dos ceps que eu pequei (na linha de cima, pega as paginas na mao, no problem 
# procurar nos raios dos endereços  que pegou   pelos 5 tipos de resestaurante mais comuns , contar, e selecionar o o menos comum


In [11]:
df.shape # At last, we will verify the rows

(36, 3)

In [12]:
    geolocator = Nominatim(user_agent="BH_explorer")
    lat= []
    long= []
    for i in range(0,36):
        # initialize your variable to None
        lat_lng_coords = None
        address = '{},Savassi, Belo Horizonte, MG, BR'.format(df.iloc[i]['Street']) # iterate across all the dataframe
        g = geolocator.geocode(address)
        if g is not None:
            lat.append (g.latitude)
            long.append (g.longitude)
        else:
            lat.append ("Drop") # Unfortunally, Geolocator don't find some of the address. 
            long.append ("Drop")
        i+=1
    df['Latitude'] = lat
    df['Longitude'] = long
    #df.drop([11,19,24,31,32],inplace=True)
    df.drop(df.index[[11,18,19,24,31,32]], inplace=True)

    df

,Street,Postcode,Neighborhood,Latitude,Longitude
0,Rua Alagoas,30130-167,Savassi,-19.9318,-43.9351
1,Rua Antônio de Albuquerque,30112-010,Savassi,-19.9395,-43.9302
2,Rua Bernardo Guimarães,30140-081,Savassi,-19.9309,-43.9333
3,Avenida Brasil,30140-008,Savassi,-19.9318,-43.9351
4,Rua Ceará,30150-314,Savassi,-19.937,-43.9297
5,Rua Cláudio Manoel,30140-105,Savassi,-19.9336,-43.9329
6,Avenida Cristóvão Colombo,30140-140,Savassi,-19.9384,-43.9353
7,Praça Diogo de Vasconcelos,30140-160,Savassi,-19.9379,-43.9356
8,Rua Fernandes Tourinho,30112-004,Savassi,-19.9398,-43.9342
9,Avenida Getúlio Vargas,30112-021,Savassi,-19.9347,-43.9294


In [13]:
# Now, we will find the latitude and longitude for Savassi
address = 'Savassi, Belo Horizonte, MG' #use the neighbourhood name
geolocator = Nominatim(user_agent="bh_explorer") #we need to create a user agent
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(df.iloc[1]['Postcode']+ ' The geograpical coordinate of Savassi-BH are {}, {}.'.format(latitude, longitude)) #at final, print the latitude and longitude to check the data;

30112-010 The geograpical coordinate of Savassi-BH are -19.9352205, -43.9344463401975.


In [14]:
# create map of belo horizonte using latitude and longitude values
map_bh = folium.Map(location=[latitude, longitude], zoom_start=15)

# add markers to map
for lat, lng, street, neighborhood in zip(df['Latitude'], df['Longitude'], df['Street'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, street)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bh)  
map_bh

In [15]:
CLIENT_ID = 'FEQ1C41QS2I5QE4OX15GPFSOIBUAD4V5B3P0IGVHHO4ZTYJU' # your Foursquare ID
CLIENT_SECRET = 'VEUH5A2VHR2YXN2UFB4D1U23FW0DZOIEORGUHHY20TXSFH5A' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [16]:
# now we will get the results using 1000 venues limit and 1000 radius, a big area but next to the neighborhood

LIMIT = 1000 # limit of number of venues returned by Foursquare API


radius = 1000 # define radius

categoryId = '4d4b7105d754a06374d81259' # filter to bring back only food main category, and the subcategorys

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&categoryId=4d4b7105d754a06374d81259&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categories={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT,
    categoryId,
    )



In [17]:
results = requests.get(url).json() #get the results of the call of url

In [18]:
# function that extracts the category of the venue, I will use one example like the new york exercise, so we will explore the item for the categories of the venues
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [19]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


### Let's create a function to repeat the same process to all the streets in neighborhood

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
        categoryId = '4d4b7105d754a06374d81259' # filter to bring back only food main category, and the subcategorys

        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&categoryId=4d4b7105d754a06374d81259&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categories={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            categoryId,)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### then we need to run the funcion for all data

In [21]:
bh_venues = getNearbyVenues(names=df['Street'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

# lets check how many venues returned

In [22]:
#let's count the Categorys by streets, to explore how to use the data
bh_venues.groupby('Venue Category').count()

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Acai House,1,1,1,1,1,1
Asian Restaurant,18,18,18,18,18,18
BBQ Joint,43,43,43,43,43,43
Baiano Restaurant,10,10,10,10,10,10
Bakery,82,82,82,82,82,82
Bistro,32,32,32,32,32,32
Brazilian Restaurant,312,312,312,312,312,312
Breakfast Spot,24,24,24,24,24,24
Buffet,59,59,59,59,59,59


In [23]:
#Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(bh_venues['Venue Category'].unique())))

There are 43 uniques categories.


In [24]:
# List the venues by the frequency. We will determinate that the category of the restaurant will be 10ª most common venue. 
#There is a lot of luck on that choice, but but the justification is that it is a choice with high demand, since it is among the main ones, 
#but at the same time it is not as common as the others. If the cluster with fewer options has this option inside, it will immediately be the next option.
bh_venues_sorted = pd.DataFrame(bh_venues.groupby('Venue Category')['Street'].count().sort_values(ascending=False))
bh_venues_sorted
#category_final = "Gastropub"

,Street
Venue Category,
Brazilian Restaurant,312
Café,220
Restaurant,219
Pizza Place,161
Snack Place,136
Burger Joint,119
Italian Restaurant,106
Vegetarian / Vegan Restaurant,92
Bakery,82


In [25]:
# one hot encoding
bh_onehot = pd.get_dummies(bh_venues[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
bh_onehot['Street'] = bh_venues['Street'] 

# move neighborhood column to the first column
fixed_columns = [bh_onehot.columns[-1]] + list(bh_onehot.columns[:-1])
bh_onehot = bh_onehot[fixed_columns]


In [26]:
### and lets examine the data size using the shape function
bh_onehot.shape

(2210, 44)

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category. 

In [27]:
bh_grouped = bh_onehot.groupby('Street').mean().reset_index()

bh_grouped

,Street,Acai House,Asian Restaurant,BBQ Joint,Baiano Restaurant,Bakery,Bistro,Brazilian Restaurant,Breakfast Spot,Buffet,Burger Joint,Café,Chinese Restaurant,Comfort Food Restaurant,Creperie,Deli / Bodega,Diner,Empada House,Falafel Restaurant,Fast Food Restaurant,Food,Food Court,Food Truck,French Restaurant,Gastropub,German Restaurant,Gluten-free Restaurant,Hawaiian Restaurant,Italian Restaurant,Japanese Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mineiro Restaurant,Pastelaria,Pizza Place,Portuguese Restaurant,Restaurant,Salad Place,Sandwich Place,Seafood Restaurant,Snack Place,Steakhouse,Sushi Restaurant,Vegetarian / Vegan Restaurant
0,Rua Pernambuco 1322,0.000000,0.000000,0.030000,0.000000,0.050000,0.010000,0.180000,0.020000,0.030000,0.040000,0.100000,0.000000,0.000000,0.010000,0.000000,0.010000,0.000000,0.000000,0.010000,0.020000,0.000000,0.020000,0.010000,0.010000,0.000000,0.000000,0.000000,0.040000,0.010000,0.000000,0.030000,0.020000,0.010000,0.040000,0.000000,0.140000,0.010000,0.020000,0.020000,0.050000,0.020000,0.010000,0.030000
1,Avenida Brasil,0.000000,0.000000,0.000000,0.000000,0.032787,0.032787,0.196721,0.016393,0.016393,0.065574,0.098361,0.000000,0.000000,0.016393,0.000000,0.016393,0.000000,0.000000,0.016393,0.016393,0.000000,0.032787,0.000000,0.000000,0.000000,0.000000,0.000000,0.032787,0.016393,0.000000,0.016393,0.032787,0.000000,0.032787,0.000000,0.147541,0.000000,0.000000,0.016393,0.065574,0.032787,0.000000,0.049180
2,Avenida Cristóvão Colombo,0.000000,0.010000,0.010000,0.010000,0.030000,0.030000,0.090000,0.010000,0.050000,0.040000,0.080000,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.030000,0.000000,0.010000,0.020000,0.020000,0.040000,0.010000,0.010000,0.000000,0.040000,0.010000,0.010000,0.020000,0.010000,0.020000,0.100000,0.000000,0.070000,0.030000,0.010000,0.000000,0.060000,0.010000,0.040000,0.050000
3,Avenida Getúlio Vargas,0.000000,0.020408,0.040816,0.000000,0.040816,0.000000,0.204082,0.000000,0.000000,0.020408,0.081633,0.020408,0.000000,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.020408,0.020408,0.000000,0.081633,0.000000,0.102041,0.000000,0.020408,0.020408,0.102041,0.040816,0.040816,0.061224
4,Praça Diogo de Vasconcelos,0.000000,0.010000,0.010000,0.010000,0.020000,0.030000,0.100000,0.010000,0.030000,0.040000,0.080000,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.030000,0.000000,0.000000,0.020000,0.020000,0.040000,0.000000,0.010000,0.000000,0.040000,0.010000,0.020000,0.020000,0.010000,0.020000,0.100000,0.000000,0.080000,0.030000,0.010000,0.000000,0.070000,0.010000,0.040000,0.060000
5,Rua Alagoas,0.000000,0.000000,0.000000,0.000000,0.032787,0.032787,0.196721,0.016393,0.016393,0.065574,0.098361,0.000000,0.000000,0.016393,0.000000,0.016393,0.000000,0.000000,0.016393,0.016393,0.000000,0.032787,0.000000,0.000000,0.000000,0.000000,0.000000,0.032787,0.016393,0.000000,0.016393,0.032787,0.000000,0.032787,0.000000,0.147541,0.000000,0.000000,0.016393,0.065574,0.032787,0.000000,0.049180
6,Rua Antônio de Albuquerque,0.000000,0.017241,0.017241,0.017241,0.068966,0.000000,0.086207,0.000000,0.000000,0.034483,0.103448,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.017241,0.000000,0.000000,0.017241,0.034483,0.051724,0.034483,0.017241,0.000000,0.000000,0.051724,0.017241,0.034483,0.017241,0.017241,0.017241,0.120690,0.000000,0.051724,0.017241,0.017241,0.017241,0.017241,0.017241,0.017241,0.017241
7,Rua Bernardo Guimarães,0.010309,0.000000,0.030928,0.000000,0.041237,0.010309,0.185567,0.020619,0.030928,0.041237,0.082474,0.000000,0.000000,0.010309,0.000000,0.010309,0.000000,0.000000,0.010309,0.020619,0.000000,0.020619,0.000000,0.010309,0.000000,0.000000,0.000000,0.041237,0.010309,0.000000,0.030928,0.020619,0.000000,0.051546,0.000000,0.144330,0.010309,0.010309,0.020619,0.051546,0.020619,0.010309,0.041237
8,Rua Ceará,0.000000,0.020833,0.000000,0.000000,0.041667,0.0

### Let's print each neighborhood along with the top 10 most common restaurants

In [28]:
num_top_venues = 10

for hood in bh_grouped['Street']:
    print("----"+hood+"----")
    temp = bh_grouped[bh_grouped['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Rua Pernambuco 1322----
                           venue  freq
0           Brazilian Restaurant  0.18
1                     Restaurant  0.14
2                           Café  0.10
3                         Bakery  0.05
4                    Snack Place  0.05
5             Italian Restaurant  0.04
6                   Burger Joint  0.04
7                    Pizza Place  0.04
8      Middle Eastern Restaurant  0.03
9  Vegetarian / Vegan Restaurant  0.03


----Avenida Brasil----
                           venue  freq
0           Brazilian Restaurant  0.20
1                     Restaurant  0.15
2                           Café  0.10
3                    Snack Place  0.07
4                   Burger Joint  0.07
5  Vegetarian / Vegan Restaurant  0.05
6                     Steakhouse  0.03
7                    Pizza Place  0.03
8             Mineiro Restaurant  0.03
9             Italian Restaurant  0.03


----Avenida Cristóvão Colombo----
                           venue  freq
0            

### Let's put that into a pandas dataframe. First, let's write a function to sort the restaurants in descending order.

In [29]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Now let's create the new dataframe and display the top 5 restaurant for each street.

In [30]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Street'] = bh_grouped['Street']

for ind in np.arange(bh_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bh_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

# To finalize, we will search for one street that doesn't contains some of the top 10 most commom restaurants in the top 10 most common venues of the street. The first will be the chosen one


,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Rua Pernambuco 1322,Brazilian Restaurant,Restaurant,Café,Snack Place,Bakery,Italian Restaurant,Pizza Place,Burger Joint,Vegetarian / Vegan Restaurant,BBQ Joint
1,Avenida Brasil,Brazilian Restaurant,Restaurant,Café,Snack Place,Burger Joint,Vegetarian / Vegan Restaurant,Mineiro Restaurant,Bakery,Bistro,Italian Restaurant
2,Avenida Cristóvão Colombo,Pizza Place,Brazilian Restaurant,Café,Restaurant,Snack Place,Vegetarian / Vegan Restaurant,Buffet,Gastropub,Burger Joint,Sushi Restaurant
3,Avenida Getúlio Vargas,Brazilian Restaurant,Snack Place,Restaurant,Café,Pizza Place,Vegetarian / Vegan Restaurant,BBQ Joint,Sushi Restaurant,Bakery,Steakhouse
4,Praça Diogo de Vasconcelos,Brazilian Restaurant,Pizza Place,Restaurant,Café,Snack Place,Vegetarian / Vegan Restaurant,Gastropub,Burger Joint,Sushi Restaurant,Italian Restaurant


# now we will Run k-means to cluster the neighborhood into 5 clusters.

In [31]:
# set number of clusters
kclusters = 5

bh_grouped_clustering = bh_grouped.drop('Street', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bh_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 2, 4, 2, 1, 3, 1, 3, 1], dtype=int32)

### Let's create a new dataframe that includes the cluster as well as the top 10 restaurants for each Street.

In [32]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bh_merged = df

# merge bh with bh_data to add latitude/longitude for each neighborhood
bh_merged = bh_merged.join(neighborhoods_venues_sorted.set_index('Street'), on='Street')

bh_merged 

,Street,Postcode,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Rua Alagoas,30130-167,Savassi,-19.9318,-43.9351,1,Brazilian Restaurant,Restaurant,Café,Snack Place,Burger Joint,Vegetarian / Vegan Restaurant,Mineiro Restaurant,Bakery,Bistro,Italian Restaurant
1,Rua Antônio de Albuquerque,30112-010,Savassi,-19.9395,-43.9302,3,Pizza Place,Café,Brazilian Restaurant,Bakery,Italian Restaurant,Restaurant,French Restaurant,Burger Joint,Gastropub,Deli / Bodega
2,Rua Bernardo Guimarães,30140-081,Savassi,-19.9309,-43.9333,1,Brazilian Restaurant,Restaurant,Café,Snack Place,Pizza Place,Vegetarian / Vegan Restaurant,Italian Restaurant,Bakery,Burger Joint,BBQ Joint
3,Avenida Brasil,30140-008,Savassi,-19.9318,-43.9351,1,Brazilian Restaurant,Restaurant,Café,Snack Place,Burger Joint,Vegetarian / Vegan Restaurant,Mineiro Restaurant,Bakery,Bistro,Italian Restaurant
4,Rua Ceará,30150-314,Savassi,-19.937,-43.9297,3,Brazilian Restaurant,Café,Pizza Place,Snack Place,Vegetarian / Vegan Restaurant,Bakery,Restaurant,Burger Joint,French Restaurant,Gastropub
5,Rua Cláudio Manoel,30140-105,Savassi,-19.9336,-43.9329,1,Brazilian Restaurant,Restaurant,Café,Burger Joint,Italian Restaurant,Vegetarian / Vegan Restaurant,Bakery,Snack Place,Pizza Place,Breakfast Spot
6,Avenida Cristóvão Colombo,30140-140,Savassi,-19.9384,-43.9353,2,Pizza Place,Brazilian Restaurant,Café,Restaurant,Snack Place,Vegetarian / Vegan Restaurant,Buffet,Gastropub,Burger Joint,Sushi Restaurant
7,Praça Diogo de Vasconcelos,30140-160,Savassi,-19.9379,-43.9356,2,Brazilian Restaurant,Pizza Place,Restaurant,Café,Snack Place,Vegetarian / Vegan Restaurant,Gastropub,Burger Joint,Sushi Restaurant,Italian Restaurant
8,Rua Fernandes Tourinho,30112-004,Savassi,-19.9398,-43.9342,2,Brazilian Restaurant,Pizza Place,Café,Restaurant,Snack Place,Burger Joint,Sushi Restaurant,Italian Restaurant,Buffet,Gastropub
9,Avenida Getúlio Vargas,30112-021,Savassi,-19.9347,-43.9294,4,Brazilian Restaurant,Snack Place,Restaurant,Café,Pizza Place,Vegetarian / Vegan Restaurant,BBQ Joint,Sushi Restaurant,Bakery,Steakhouse


In [33]:
#Then, just to check the most overflowing cluster, I will count the numbers .groupby('Venue Category')['Street'].count().sort_values(ascending=False))
bh_merged.groupby('Cluster Labels')['Street'].count().sort_values(ascending=False)

Cluster Labels
1    10
2     7
3     6
0     6
4     1
Name: Street, dtype: int64

## Finally, let's visualize the resulting clusters

In [34]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=15)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bh_merged['Latitude'], bh_merged['Longitude'], bh_merged['Neighborhood'], bh_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [35]:
# por fim vou escolher na mão o 11 valor mais escolhido e colocar no cluster 11